In [1]:
from bs4 import BeautifulSoup
import pandas as pd

# seleinum imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = "http://novostroykirf.ru/1/#!devsearch;page=1"
browser = webdriver.PhantomJS()
browser.get(url)
element = WebDriverWait(browser, 5).until(
    EC.presence_of_element_located((By.XPATH, "html/body/div[4]/div[1]/div[5]/div/div/div[4]/div/div[3]/div/div[2]/div/div/table/tbody/tr[16]/td[1]/div/div/a"))
)

In [3]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [4]:
# get columns names
column_names = [col_tag.text for col_tag in soup.find('thead').find_all('th')]

In [5]:
# fill with main data from table
table = soup.find_all('table')[1].find('tbody')
table_rows = table.find_all('tr')
list_rows = []
for row in table_rows:
    company_info = []
    for cell in row.find_all('td'):
        # for some encoding issues
        company_info.append(cell.text.replace('\xa0', ' '))
    list_rows.append(company_info)

In [6]:
# create DataFrame
df = pd.DataFrame(list_rows, columns=column_names)

In [7]:
df.head()

,Застройщик,Количество объектов,"Объем строительства, m2",Рейтинг РАСК
0,"ООО Мир, г. Чита",2,13 407,Г1
1,"ООО Новый мир плюс, г. Владимир",12,111 564,Б1
2,"ООО УКС КЧУС, г. Кирово-Чепецк, бренд: Холдинг...",4,37 328,В2
3,"ООО Кировспецмонтаж, г. Киров",12,146 911,Б1
4,"ООО СК СибЛидер, г. Красноярск",7,60 842,Б2


In [8]:
# Split on ','
df_first = df.iloc[:, 0].str.split(',', expand=True)
df_first.head()

,0,1,2
0,ООО Мир,г. Чита,None
1,ООО Новый мир плюс,г. Владимир,None
2,ООО УКС КЧУС,г. Кирово-Чепецк,бренд: Холдинг Кирово-Чепецкое управление стр...
3,ООО Кировспецмонтаж,г. Киров,None
4,ООО СК СибЛидер,г. Красноярск,None


In [9]:
# Extract companies and cities
col_company = df_first.iloc[:, 0]
col_city = df_first.apply(lambda x: x[x.str.contains('г\.').idxmax()], axis=1)

col_company.name = 'Застройщик'
col_city.name = "Город"

In [10]:
# concatenate for final DataFrame
df_final = pd.concat([col_company, col_city, df.iloc[:, 1:]], axis=1)

In [11]:
df_final.head()

,Застройщик,Город,Количество объектов,"Объем строительства, m2",Рейтинг РАСК
0,ООО Мир,г. Чита,2,13 407,Г1
1,ООО Новый мир плюс,г. Владимир,12,111 564,Б1
2,ООО УКС КЧУС,г. Кирово-Чепецк,4,37 328,В2
3,ООО Кировспецмонтаж,г. Киров,12,146 911,Б1
4,ООО СК СибЛидер,г. Красноярск,7,60 842,Б2


In [12]:
# clear from spaces and convert to int
df_final.iloc[:, -2] = df_final.iloc[:, -2].str.replace('\s+', '')
df_final.loc[:, 'Количество объектов'] = df_final.loc[:, 'Количество объектов'].astype(int)
df_final.loc[:, 'Объем строительства, m2'] = df_final.loc[:, 'Объем строительства, m2'].astype(int)

In [13]:
df_final.dtypes

Застройщик                 object
Город                      object
Количество объектов         int64
Объем строительства, m2     int64
Рейтинг РАСК               object
dtype: object

In [14]:
df_final

,Застройщик,Город,Количество объектов,"Объем строительства, m2",Рейтинг РАСК
0,ООО Мир,г. Чита,2,13407,Г1
1,ООО Новый мир плюс,г. Владимир,12,111564,Б1
2,ООО УКС КЧУС,г. Кирово-Чепецк,4,37328,В2
3,ООО Кировспецмонтаж,г. Киров,12,146911,Б1
4,ООО СК СибЛидер,г. Красноярск,7,60842,Б2
5,ООО УСК Сибиряк,г. Красноярск,13,188190,Г1
6,ЗАО Строительная фирма Трест-5,г. Омск,4,33492,В1
7,ООО Фирма Кристина,г. Ростов-на-Дону,4,9789,Б3
8,ЗАО Краснодарпроектстрой,г. Краснодар,1,10532,Г3
9,ООО ЮгСпецСтрой,г. Ростов-на-Дону,2,34568,В1


In [15]:
df_final.to_csv('task3_cleared.csv', sep=';', index=False)